In [1]:
#import sparql_dataframe
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

In [ ]:
""" {
    ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION { """

## `writers_and_works`

#### It retrieves all italian authors of written works having one or more work linked on wikidata

In [90]:
writers_and_works_query = """
SELECT DISTINCT ?person ?personLabel ?authorImdbId
(COUNT(DISTINCT ?work) AS ?work)
(COUNT(DISTINCT ?workCreated) AS ?creatorP) (COUNT(DISTINCT ?workAuthored) AS ?authorP) (COUNT(DISTINCT ?workInAuthorsLists) AS ?workInAuthorsLists)
(COUNT(DISTINCT ?occupationWriter) AS ?writer) (COUNT(DISTINCT ?occupationNovelist) AS ?novelist) (COUNT(DISTINCT ?occupationPoet) AS ?poet) (COUNT(DISTINCT ?occupationProsaist) AS ?prosaist) (COUNT(DISTINCT ?occupationScreenwriter) AS ?screenwriter)

WHERE {
    {
      ?person ?occupationWriter  wd:Q36180.
    }UNION{
      ?person ?occupationNovelist  wd:Q6625963
    }UNION{
      ?person ?occupationPoet  wd:Q49757
    }UNION{
      ?person ?occupationProsaist  wd:Q12144794
    }UNION{
      ?person ?occupationScreenwriter wd:Q28389
    }
    
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 ?mcItaly .


    {
      ?work wdt:P170 ?person .
      BIND (?work AS ?workCreated)
     }UNION{
      ?work wdt:P50 ?person .                             #attenzione: mi connette ad ogni opera, quindi se uno è segnato come writer ma è anchre pittore avrò anche le sue opere da pittore
      BIND (?work AS ?workAuthored)
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
      BIND (?work AS ?workInAuthorsLists)
    }
    
    {                                                  #more efficient
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    #{                                                    #less efficient
    #  VALUES ?writtenWork {wd:Q571 wd:Q3331189}          #VALUES ?writtenWork {wd:Q47461344 wd:Q571 wd:Q3331189 wd:Q7725634 } . ?work wdt:P31 ?writtenWork  non funziona
    #  ?work wdt:P31 ?writtenWork .
    #}UNION{
    #  VALUES ?writtenWork {wd:Q47461344 wd:Q7725634}
    #  ?work wdt:P31 ?writtenWork .
    #}
  
  
  OPTIONAL {?person wdt:P345 ?authorImdbId}
 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?person ?personLabel ?authorImdbId
"""

ww_df = query_wikidata(endpoint, writers_and_works_query, user_agent, True).fillna("")
ww_df

,person,personLabel,authorImdbId,work,creatorP,authorP,workInAuthorsLists,writer,novelist,poet,prosaist,screenwriter
0,http://www.wikidata.org/entity/Q25163,Roberto Saviano,nm2500207,6,0,6,3,1,0,0,0,1
1,http://www.wikidata.org/entity/Q23018547,Paolo Terni,,1,0,1,0,1,0,0,0,0
2,http://www.wikidata.org/entity/Q3840490,Luisito Bianchi,,1,0,1,0,1,0,0,0,0
3,http://www.wikidata.org/entity/Q1090676,Cesare Garboli,,3,0,3,0,1,0,0,0,0
4,http://www.wikidata.org/entity/Q15711154,Giuseppe Andrea Rocca,,1,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1338,http://www.wikidata.org/entity/Q453614,Alba de Céspedes y Bertini,nm0207968,2,0,2,0,1,1,1,0,1
1339,http://www.wikidata.org/entity/Q105036845,Vinci Corsi,nm12087141,1,0,1,1,1,0,0,0,0
1340,http://www.wikidata.org/entity/Q3848969,Mario Pincherle,,2,0,2,0,1,0,0,0,0
1341,http://www.wikidata.org/entity/Q3856713,Michele Monina,,1,0,1,0,1,0,0,0,0


## `writers_works_and_derivatives`

#### It retrieves all italian authors of written works that are linked to one of their work on wikidata, which are in turn linked to some derovative work on wikidata

In [100]:
ww_and_derivatives_query = """
SELECT DISTINCT ?person ?personLabel ?authorImdbId
(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
(COUNT(DISTINCT ?influencedWork) AS ?influencedByP) (COUNT(DISTINCT ?workBasedOn) AS ?basedOnP) (COUNT(DISTINCT ?referencedWork) AS ?referencesWorkP) (COUNT(DISTINCT ?derivedWork) AS ?derivedWorkP) (COUNT(DISTINCT ?inspiredWork) AS ?inspiredByP)  (COUNT(DISTINCT ?workAfter) AS ?afterWorkByP)

WHERE {
    
    {
      ?person wdt:P106  wd:Q36180.
    }UNION{
      ?person wdt:P106  wd:Q6625963
    }UNION{
      ?person wdt:P106  wd:Q49757
    }UNION{
      ?person wdt:P106  wd:Q12144794
    }UNION{
      ?person wdt:P106 wd:Q28389
    }
    
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 ?mcItaly .


    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                             #attenzione: mi connette ad ogni opera, quindi se uno è segnato come writer ma è anchre pittore avrò anche le sue opere da pittore
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }
    
    {                                                  #more efficient
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    #{                                                    #less efficient
    #  VALUES ?writtenWork {wd:Q571 wd:Q3331189}          #VALUES ?writtenWork {wd:Q47461344 wd:Q571 wd:Q3331189 wd:Q7725634 } . ?work wdt:P31 ?writtenWork  non funziona
    #  ?work wdt:P31 ?writtenWork .
    #}UNION{
    #  VALUES ?writtenWork {wd:Q47461344 wd:Q7725634}
    #  ?work wdt:P31 ?writtenWork .
    #}


    {
      ?derivativeWork wdt:P737 ?work . 
      BIND(?derivativeWork AS ?influencedWork)
    }UNION{
      ?derivativeWork wdt:P144 ?work .
      BIND(?derivativeWork AS ?workBasedOn)
    }UNION{
      ?derivativeWork wdt:P941 ?work .
      BIND(?derivativeWork AS ?inspiredWork)
    }UNION{  
      ?derivativeWork wdt:P8371 ?work .
      BIND(?derivativeWork AS ?referencedWork)
    }UNION{
      ?work wdt:P4969 ?derivativeWork .
      BIND(?derivativeWork AS ?derivedWork)
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
      BIND(?derivativeWork AS ?workAfter)
    }
    

  
  
  OPTIONAL {?person wdt:P345 ?authorImdbId}
 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?person ?personLabel ?authorImdbId
"""

wwd_df = query_wikidata(endpoint, ww_and_derivatives_query, user_agent, True).fillna("")
wwd_df

,person,personLabel,authorImdbId,work,derivativeWork,influencedByP,basedOnP,referencesWorkP,derivedWorkP,inspiredByP,afterWorkByP
0,http://www.wikidata.org/entity/Q271814,Giovanni Verga,nm0894015,2,4,0,4,0,4,0,0
1,http://www.wikidata.org/entity/Q5592,Michelangelo,,68,3,0,0,0,0,0,3
2,http://www.wikidata.org/entity/Q3036220,Donato Carrisi,nm2621810,1,1,0,1,0,0,0,0
3,http://www.wikidata.org/entity/Q3767950,Giovanni Pirelli,,1,1,0,1,0,0,0,0
4,http://www.wikidata.org/entity/Q3713208,Domenico Rea,,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
114,http://www.wikidata.org/entity/Q741862,Massimo Bontempelli,nm0095219,1,1,0,1,0,0,0,0
115,http://www.wikidata.org/entity/Q3680870,Clizia Gurrado,,1,1,0,1,0,1,0,0
116,http://www.wikidata.org/entity/Q368127,Elena Ferrante,nm0273919,12,4,0,4,0,4,0,1
117,http://www.wikidata.org/entity/Q25120,Pier Paolo Pasolini,nm0001596,18,2,0,0,0,0,0,2


In [94]:
wwd_df.sum()

person             http://www.wikidata.org/entity/Q271814http://w...
personLabel        Giovanni VergaMichelangeloDonato CarrisiGiovan...
authorImdbId       nm0894015nm2621810nm0090504nm0019604nm0092435n...
work                                                            1265
derivativeWork                                                   451
influencedByP                                                      5
basedOnP                                                         330
referencesWorkP                                                    3
derivedWorkP                                                     128
inspiredByP                                                       23
afterWorkByP                                                     146
dtype: object

## `derivatives_type`

#### Retrieves the type of derivative works connected with an author's works

In [124]:
derivatives_type = """
SELECT ?derivativeWorkClassLabel (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork) 
WHERE {
    
    {
      ?person wdt:P106  wd:Q36180
    }UNION{
      ?person wdt:P106  wd:Q6625963
    }UNION{
      ?person wdt:P106  wd:Q49757
    }UNION{
      ?person wdt:P106  wd:Q12144794
    }UNION{
      ?person wdt:P106 wd:Q28389 
    }
    
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}   #modern/contemporary Italy can be one of these entities
    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 ?mcItaly .


    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                             #attenzione: mi connette ad ogni opera, quindi se uno è segnato come writer ma è anchre pittore avrò anche le sue opere da pittore
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }
    
    {                                                  #28s
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    

    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969}    #wdt:P941 wdt:P8371 wdt:P4969 wdt:P1877
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
    }
   


    ?derivativeWork wdt:P31 ?derivativeWorkClass .

    ?derivativeWorkClass rdfs:label ?derivativeWorkClassLabel .
    FILTER((LANG(?derivativeWorkClassLabel)) = "en")
    


 
                              
} 
GROUP BY ?derivativeWorkClassLabel 
"""

dt_df = query_wikidata(endpoint, derivatives_type, user_agent, True).fillna("")
dt_df

,derivativeWorkClassLabel,derivativeWork
0,teleplay,12
1,film,140
2,theatrical production,49
3,poem,4
4,television film,12
...,...,...
60,television program,2
61,video game project,1
62,work,1
63,tetralogy,1
